# Projeto 2 - Ciência dos Dados

Rodrigo Sennati Mattar 

Lucca Nazari

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [5]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [6]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Luca Nazari]***

In [7]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [8]:
#Produto escolhido:
produto = 'airpods'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 250

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [9]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(list(set(msgs)))

KeyboardInterrupt: 

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes



In [57]:
import pandas as pd
import numpy as np

from IPython.display import display

pd.options.display.max_rows = 13 # Escolher quantos prints as funções do pandas fazem

In [58]:
import re 

def cleanup(text):
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [59]:
tweets = pd.read_excel('airpods1.xlsx')

In [82]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
from pandas import DataFrame
from numpy import arange
from numpy import percentile
import numpy as np
from matplotlib.font_manager import FontProperties
import string

In [103]:
# FILTRANDO TODOS OS TWEETS RELEVENATES
relev = tweets.loc[:,['Treinamento','Relevante']]
#FILTRANDO OS TWEETS DE ACORDO COM SUA RELEVANCIA (=1,=2,=3)
relev1 = tweets.loc[tweets.Relevante==1]
relev2 = tweets.loc[tweets.Relevante==2]
relev3 = tweets.loc[tweets.Relevante==3]
texto_relev1_ = relev1.Treinamento
texto_relev2_ = relev2.Treinamento
texto_relev3_ = relev3.Treinamento

#SEPARANDO CADA GRUPO DE RELEVANCIA PARA UM ARQUIVO CSV
texto_relev1_.to_csv('relev1.csv', header=True, index=False, encoding='utf-8')
texto_relev2_.to_csv('relev2.csv', header=True, index=False, encoding='utf-8')
texto_relev3_.to_csv('relev3.csv', header=True, index=False, encoding='utf-8')

In [112]:
#ABRINDO OS ARQUIVOS CSV
#primeiro arquivo(relevancia =1)
df1 = pd.read_csv('relev1.csv')
df1 = df1.dropna(axis=0, how = 'any')
#LIMPANDO O TEXTO DE CADA ARQUIVO CSV E DANDO VALUE COUNTS 
def removetext(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

df1['Treinamento'] = df1['Treinamento'].apply(removetext)


relev1_array = df1['Treinamento'].str.split(' ', expand=True).stack().value_counts(True)

#segund arquivo(relevancia = 2)
df2 = pd.read_csv('relev2.csv')
df2 = df2.dropna(axis=0, how = 'any')
df2['Treinamento'] = df2['Treinamento'].apply(removetext)
relev2_array = df2['Treinamento'].str.split(' ', expand=True).stack().value_counts(True)

#terceiro arquivo (relevancia = 3)
df3 = pd.read_csv('relev3.csv')
df3 = df3.dropna(axis=0, how = 'any')
df3['Treinamento'] = df3['Treinamento'].apply(removetext)
relev3_array = df2['Treinamento'].str.split(' ', expand=True).stack().value_counts(True)

In [111]:
rele

1      0.746807
2      0.113139
3      0.047445
4      0.023266
6      0.013230
5      0.009580
         ...   
19     0.000456
15     0.000456
145    0.000456
111    0.000456
150    0.000456
323    0.000456
Length: 46, dtype: float64

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**